In [ ]:
# default_exp core.camera

In [ ]:
#hide
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Camera
> A camera file with a range of functions to handle the camera and it's coordinates.

In [ ]:
#export
import numpy as np
import torch
from runningpose.core.utils import wrap
from runningpose.core.quaternion import qrot, qinverse

In [ ]:
#export
def normalize_screen_coordinates(X, w, h): 
    """
    Normalize so that [0, w] is mapped to [-1, 1], while preserving the 
    aspect ratio.
    """
    assert X.shape[-1] == 2

    return X/w*2 - [1, h/w]

In [ ]:
#export
def image_coordinates(X, w, h):
    """Return the image coordinates"""
    assert X.shape[-1] == 2
    # Reverse camera frame normalization
    return (X + [1, h/w])*w/2

In [ ]:
#export
def world_to_camera(X, R, t):
    """
    Converts from world to camera coordinates with Human3.6M extrinsic 
    camera parameters structure.
    """
    Rt = wrap(qinverse, R) # Invert rotation
    # Rotate and translate
    return wrap(qrot, np.tile(Rt, (*X.shape[:-1], 1)), X - t) 

In [ ]:
#export
def camera_to_world(X, R, t):
    """
    Converts from camera to world coordinates with Human3.6M extrinsic 
    camera parameters structure.
    """
    # Rotate and translate
    return wrap(qrot, np.tile(R, (*X.shape[:-1], 1)), X) + t 

In [ ]:
#export
def project_to_2d(X, camera_params):
    """
    Project 3D points to 2D using the Human3.6M camera projection 
    function.
    This is a differentiable and batched reimplementation of the 
    original MATLAB script.
    
    Arguments:
    X -- 3D points in *camera space* to transform (N, *, 3)
    camera_params -- intrinsic parameteres (N, 2+2+3+2=9)
    """
    assert X.shape[-1] == 3
    assert len(camera_params.shape) == 2
    assert camera_params.shape[-1] == 9
    assert X.shape[0] == camera_params.shape[0]
    
    while len(camera_params.shape) < len(X.shape):
        camera_params = camera_params.unsqueeze(1)
        
    f = camera_params[..., :2]
    c = camera_params[..., 2:4]
    k = camera_params[..., 4:7]
    p = camera_params[..., 7:]
    
    XX = torch.clamp(X[..., :2] / X[..., 2:], min=-1, max=1)
    r2 = torch.sum(XX[..., :2]**2, dim=len(XX.shape)-1, keepdim=True)

    radial = 1 + torch.sum(k * torch.cat((r2, r2**2, r2**3), 
                           dim=len(r2.shape)-1), 
                           dim=len(r2.shape)-1, 
                           keepdim=True)
    tan = torch.sum(p*XX, dim=len(XX.shape)-1, keepdim=True)

    XXX = XX*(radial + tan) + p*r2
    
    return f*XXX + c

In [ ]:
#export
def project_to_2d_linear(X, camera_params):
    """
    Project 3D points to 2D using only linear parameters 
    (focal length and principal point).
    
    Arguments:
    X -- 3D points in *camera space* to transform (N, *, 3)
    camera_params -- intrinsic parameteres (N, 2+2+3+2=9)
    """
    assert X.shape[-1] == 3
    assert len(camera_params.shape) == 2
    assert camera_params.shape[-1] == 9
    assert X.shape[0] == camera_params.shape[0]
    
    while len(camera_params.shape) < len(X.shape):
        camera_params = camera_params.unsqueeze(1)
        
    f = camera_params[..., :2]
    c = camera_params[..., 2:4]
    
    XX = torch.clamp(X[..., :2] / X[..., 2:], min=-1, max=1)
    
    return f*XX + c

## Miqus Camera
> Functions that handles extrinsic camera parameters from Qualisys calibration file 

In [ ]:
#export
def world_to_camera_miqus(P, R, T):
  """
  Convert points from world to camera coordinates
  Args
    P: Nx3 3d points in world coordinates
    R: 3x3 Camera rotation matrix
    T: 3x1 Camera translation parameters
  Returns
    X_cam: Nx3 3d points in camera coordinates
  """

  assert len(P.shape) == 2
  assert P.shape[1] == 3

  X_cam = R.dot( P.T - T ) # rotate and translate

  return X_cam.T

In [ ]:
#export
def camera_to_world_miqus(P, R, T):
  """Inverse of world_to_camera_frame
  Args
    P: Nx3 points in camera coordinates
    R: 3x3 Camera rotation matrix
    T: 3x1 Camera translation parameters
  Returns
    X_cam: Nx3 points in world coordinates
  """

  assert len(P.shape) == 2
  assert P.shape[1] == 3

  X_cam = R.T.dot( P.T ) + T # rotate and translate

  return X_cam.T

In [ ]:
#export
def project_point_radial(P, R, T, f, c, k, p):
  """
  Project points from 3d to 2d using camera parameters
  including radial and tangential distortion
  Args
    P: Nx3 points in world coordinates
    R: 3x3 Camera rotation matrix
    T: 3x1 Camera translation parameters
    f: (scalar) Camera focal length
    c: 2x1 Camera center
    k: 3x1 Camera radial distortion coefficients
    p: 2x1 Camera tangential distortion coefficients
  Returns
    Proj: Nx2 points in pixel space
    D: 1xN depth of each point in camera space
    radial: 1xN radial distortion per point
    tan: 1xN tangential distortion per point
    r2: 1xN squared radius of the projected points before distortion
  """

  # P is a matrix of 3-dimensional points
  assert len(P.shape) == 2
  assert P.shape[1] == 3

  N = P.shape[0]
  X = R.dot(P.T - T) # rotate and translate
  XX = X[:2,:] / X[2,:]
  r2 = XX[0,:]**2 + XX[1,:]**2

  radial = 1 + np.einsum('ij,kl->il', np.tile(k,(1, N)), np.array([r2, r2**2, r2**3]))
  tan = p[0]*XX[1,:] + p[1]*XX[0,:]

  XXX = XX * np.tile(radial+tan,(2,1)) + np.outer(np.array([p[1], p[0]]).reshape(-1), r2)

  Proj = (f * XXX) + c
  Proj = Proj.T

  D = X[2,]

  return Proj, D, radial, tan, r2

In [ ]:
from nbdev.export import notebook2script; notebook2script()

Converted 00_model.ipynb.
Converted 01_loss.ipynb.
Converted 02_skeleton.ipynb.
Converted 03_mocap_dataset.ipynb.
Converted 04_h36m_dataset.ipynb.
Converted 05_camera.ipynb.
Converted 06_quaternion.ipynb.
Converted 07_utils.ipynb.
Converted 08_generators.ipynb.
Converted 09_custom_dataset.ipynb.
Converted 10_visualization.ipynb.
Converted 11_arguments.ipynb.
Converted 12_data_utils.ipynb.
Converted 13_prepare_data_2d_custom.ipynb.
Converted 14_infer_video.ipynb.
Converted 15_prepare_data_COCO.ipynb.
Converted 16_pycococreatortools.ipynb.
Converted 17_format_qtmdata.ipynb.
Converted 18_runningpose_dataset.ipynb.
No export destination, ignored:
#export
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# Import some common libraries
import numpy as np
import matplotlib.pyplot as plt
import os, json, cv2, random

# Import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine im

AttributeError: 'NoneType' object has no attribute 'start'